Haal gebruikers op die films hebben gerate. Er ontstaat een RDD[(userId, movieId, rating)].

In [21]:
userdata = sc.textFile("../../../schooldata/nieuw/ratings_train.csv")

firstline = userdata.first()
userdata = userdata.filter(lambda x: x != firstline)

data_normalized = userdata.map(lambda x: x.split("::"))
user_mov_rating = data_normalized.map(lambda x: (x[0], x[1], x[2])).cache()

user_mov_rating.take(5)

[('11973', '11', '3.0'),
 ('11973', '14', '4.0'),
 ('11973', '32', '2.0'),
 ('11973', '36', '3.0'),
 ('11973', '110', '5.0')]

Haal films op uit de database en map ze in een RDD zodat er een RDD[(movieId, title)] ontstaat.

In [4]:
moviedata = sc.textFile("../../../schooldata/nieuw/movies.csv")

firstline = moviedata.first()
movies_normalized = moviedata.filter(lambda x: x != firstline)\
.map(lambda x: x.split(','))

mov_id_name = movies_normalized.map(lambda x: (x[0], x[1])).cache()

mov_id_name.take(5)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)'),
 ('4', 'Waiting to Exhale (1995)'),
 ('5', 'Father of the Bride Part II (1995)')]

Initialiseer de benodigde ALS parameters en laat ALS trainen. Vervolgens een voorspelde lijst in predictions stoppen, die we laten joinen met de test data split.

In [38]:
from pyspark.mllib.recommendation import ALS
import math

# cast user_mov_rating tuple values ((int, int) float).
every_rating = user_mov_rating.map(lambda x: (int(x[0]), int(x[1]), float(x[2]))).cache()

# verdeel rdd data 7/totaal in cls en 3/totaal in test.
cls, test = every_rating.randomSplit([7, 3], seed=None)

# laat ALS het cls rdd trainen.
trained_prototype = ALS.train(cls, rank = 10, seed = 5, iterations = 5, lambda_ = 0.01)

test_predictions = test.map(lambda x: (x[0], x[1]))

# de rating wordt voorspeld voor userId en movieId door het getrainde ALS prototype 
# en word gemapt als ((userId, movieId), rating).
predictions = trained_prototype.predictAll(test_predictions)\
.map(lambda x: ((x[0], x[1]), x[2]))

# de test rdd joint de predictions rdd en dan krijgen we ((userId, movieId), (rating, rating)).
rnp = test.map(lambda x: ((int(x[0]), int(x[1])), float(x[2])))\
.join(predictions)

# bereken de Root Mean Square Deviation bij de rnp data.
error = math.sqrt(rnp.map(lambda x: (x[1][0] - x[1][1])**2).mean())

print('Met rank %s is de RMSE: %s' % (rank, error))

Met rank 10 is de RMSE: 0.8181872349401217


In [39]:
predictions.take(5)

[((18624, 71448), 4.377309603487989),
 ((18624, 2528), 3.1917373043772908),
 ((18624, 1232), 4.22055941615261),
 ((18624, 1208), 4.585495656979816),
 ((18624, 4641), 3.775603885951872)]

In [40]:
rnp.take(5)

[((73378, 647), (3.0, 3.2620855157585034)),
 ((74565, 457), (3.0, 3.6568727354560835)),
 ((41368, 5459), (1.5, 2.57529339406996)),
 ((3157, 40815), (1.0, 3.5766695054816955)),
 ((122862, 7445), (4.0, 4.1170104414178255))]

Werk het movie aanbevelingsalgoritme uit, zodat users die bepaalde movies nog niet gezienheb